In [1]:
!pip install emoji==0.6.0
!pip install transformers
!pip install soynlp
!pip install pytorch_lightning
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import transformers
import emoji
import soynlp
import pytorch_lightning

In [3]:
import os
import pandas as pd
import numpy as np

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

In [4]:
from utils.emotion_model import Model
from utils.infer_emotion import infer
from utils.load_data import load_dataset, idx2emoticon, emoticon2idx, convert_sentence_emotion, add_emotion

In [5]:
from sentence_transformers import SentenceTransformer, util

from sklearn.datasets import make_classification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
emotion_model_path = '/content/drive/MyDrive/인공지능프로젝트/team_project/val_acc0.7202.ckpt'
sentence_model_path = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
emoticon_sentence_path = '/content/sentence_emoticon.txt'
idx2emoticon_path = '/content/idx2emoticon.txt'
emoticon2idx_path = '/content/emoticon2idx.txt'

In [8]:
idx2emoticon = idx2emoticon(idx2emoticon_path)
emoticon2idx = emoticon2idx(emoticon2idx_path)

In [9]:
emotion_model = Model.load_from_checkpoint(emotion_model_path)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.de

In [10]:
sentence_model = SentenceTransformer(sentence_model_path)

In [36]:
train_sentences, train_emotions, train_labels = load_dataset(emoticon_sentence_path)
train_sentences = convert_sentence_emotion(sentence_model, train_emotions, train_sentences)

In [37]:
knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute', metric='cosine')
knn.fit(train_sentences, train_labels)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_neighbors=3)

In [103]:
input_sentence = ['오늘 아침이 좀 개운한데?']

In [104]:
sentence_emotion = infer(emotion_model, input_sentence)
sentence_emotion = np.argmax(sentence_emotion.cpu().detach())
sentence_emotion

tensor(2)

In [105]:
sentence_vector = add_emotion(sentence_model, int(sentence_emotion), input_sentence)

In [106]:
emoticon_idx = knn.predict([sentence_vector])

In [107]:
print('Recommended emoticon: ' + idx2emoticon[int(emoticon_idx)])

Recommended emoticon: ⸜(*ˊᗜˋ*)⸝
